Import necessary packages

In [ ]:
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


Prepare Classifier and read the data into a datframe

In [ ]:
mlp = MLPClassifier(max_iter=100)
df = pd.read_feather('../data/processed/team_score_and_buy__dataset__training.feather')

Configure all parameters to test for our Hyperparameter Tuning

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,),(20, 10)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

Prepare, normalize and encode the data

In [ ]:
label_encoder = preprocessing.LabelEncoder()
categorical_value_columns = [
    'buy_type', 'starting_side', 'current_side']

unnecessary_columns = ['match_name', 'client_name', 'round_end_reason', 'round_win', 'team',
                        'map_name',
                        'round_start_eq_val', 'spend', '_data_id', '_match_id']

df.drop(labels=unnecessary_columns, inplace=True, axis=1)

for c in categorical_value_columns:
  df[c] = label_encoder.fit_transform(df[c])

col = df.columns

standscl = preprocessing.StandardScaler()
standscl.fit(df)
standscl.transform(df)
df = standscl.transform(df)
df = pd.DataFrame(df, columns=col)

df['match_win'] = label_encoder.fit_transform(df['match_win'])
X = df.drop('match_win', axis=1)
y = df['match_win']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

Initialize our GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

Return our best parameters and results from the Tuning Grid Search

In [ ]:
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

See the best results

In [ ]:
y_true, y_pred = y_test , clf.predict(X_test)

from sklearn.metrics import classification_report

print('Results on the test set:')
print(classification_report(y_true, y_pred))